In [1]:
import numpy as np
import pandas as pd

path_prefix = '/home/jaySandesara/NSBI_workflow_tutorial/FAIR_universe_Higgs_tautau/'

import os
os.chdir(f"{path_prefix}")

from HiggsML.datasets import download_dataset

In [2]:
data = download_dataset("https://zenodo.org/records/15131565/files/FAIR_Universe_HiggsML_data.zip")

2025-05-31 17:09:12,295 - HiggsML.datasets     - INFO     - Handling as URL: https://zenodo.org/records/15131565/files/FAIR_Universe_HiggsML_data.zip
2025-05-31 17:09:12,296 - HiggsML.datasets     - INFO     - Current working directory: /data/jaySandesara/NSBI_workflow_tutorial/FAIR_universe_Higgs_tautau
2025-05-31 17:09:12,384 - HiggsML.datasets     - INFO     - Total rows: 220099101
2025-05-31 17:09:12,384 - HiggsML.datasets     - INFO     - Test size: 66029730


In [13]:
# Assign an integer with each of the processes (same as the dictionary used to load)
labels_dict = {"htautau": 0,
              "ztautau":1,
              "ttbar":2,
              "diboson":3}

np.save(f"{saved_data}labels_dictionary.npy", labels_dict)


In [ ]:
data.load_train_set(train_size=0.05)
df_training = data.get_train_set()

2025-05-31 17:09:19,217 - HiggsML.datasets     - INFO     - Selected train size: 7703468
2025-05-31 17:11:59,890 - HiggsML.datasets     - INFO     - Data loaded successfully


In [5]:
df_training.columns

Index(['PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_had_pt', 'PRI_had_eta',
       'PRI_had_phi', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta',
       'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
       'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_n_jets',
       'PRI_jet_all_pt', 'PRI_met', 'PRI_met_phi', 'weights',
       'detailed_labels', 'labels', 'DER_mass_transverse_met_lep',
       'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet',
       'DER_prodeta_jet_jet', 'DER_deltar_had_lep', 'DER_pt_tot', 'DER_sum_pt',
       'DER_pt_ratio_lep_had', 'DER_met_phi_centrality',
       'DER_lep_eta_centrality'],
      dtype='object')

In [6]:
path_prefix = '/home/jaySandesara/NSBI_workflow_tutorial/FAIR_universe_Higgs_tautau/'

# Path for saving interemdiate objects, like NN predictions
saved_data = f'{path_prefix}cached_data/'


if not os.path.exists(saved_data):
    print(f"Making new directory for caching data {saved_data}")
    os.makedirs(saved_data)


In [8]:
df_training.detailed_labels.unique()

array(['ztautau', 'htautau', 'ttbar', 'diboson'], dtype=object)

In [10]:
df_training['train_labels'] = -1 

for key in labels_dict:
    df_training.loc[df_training['detailed_labels'] == key, 'train_labels'] = labels_dict[key]


In [15]:
df_training.to_hdf(f"{saved_data}df_inclusive.h5", key="dataset", mode="w")

In [16]:
columns = ['PRI_lep_pt', 'PRI_lep_eta', 'PRI_met', 'PRI_met_phi', 'PRI_n_jets']

histograms = {}
for feature in columns:
    histograms[feature] = {}
    for count, label in enumerate(labels_dict):

        if count==0:
            mask_process = (df_training.detailed_labels == label)
            histograms[feature][label], bins = np.histogram(
                features[:, data_dict[feature]][mask_process], 
                weights=weights[mask_process], 
                bins=nbins
            )
        else:
            mask_process = (df_training.detailed_labels == label)
            histograms[feature][label], _ = np.histogram(
                features[:, data_dict[feature]][mask_process], 
                weights=weights[mask_process], 
                bins=bins
            )
            
        print(histograms[feature][label])

NameError: name 'features' is not defined

In [ ]:
# Plot the training features to be used

bins_dict = {
    'PRI_lep_pt': np.linspace(10.0, 500.0, num=20),
    'PRI_lep_eta': np.linspace(-2.5, 2.5, num=20),
    'lep_phi': np.linspace(-2.5, 2.5, num=20),
    'H_T': np.linspace(50.0, 2000.0, num=20),
    'n_jets': np.linspace(-0.5, 4.5, num=6)
}

data_dict = {
    'lep_pt': 0,  # Column index for lep_pt
    'lep_eta': 1, # Column index for lep_eta
    'lep_phi': 2, # Column index for lep_phi
    'H_T': 3,     # Column index for H_T
    'n_jets': 4       # Column index for n_jets
}

xlabel_dict = {
    'lep_pt': 'lepton $p_T$ [GeV]', 
    'lep_eta': 'lepton $\eta$ [GeV]', 
    'lep_phi': 'lepton $\phi$ [GeV]', 
    'H_T': '$H_T$ [GeV]',     
    'n_jets': '$n_{jets}$'      
}

histograms = {}
for key, bin_edges in bins_dict.items():
    histograms[key] = {}
    for label in labels_dict:
        mask_process = (train_labels == labels_dict[label])
        histograms[key][label], _ = np.histogram(
            features[:, data_dict[key]][mask_process], 
            weights=weights[mask_process], 
            bins=bin_edges
        )
        print(histograms[key][label])

histograms_var_pT_res = {}
for key, bin_edges in bins_dict.items():
    histograms_var_pT_res[key] = {}
    for label in labels_dict:
        mask_process = (train_labels == labels_dict[label])
        histograms_var_pT_res[key][label], _ = np.histogram(
            features[:, data_dict[key]][mask_process], 
            weights=weights[mask_process], 
            bins=bin_edges
        )


palette = plt.rcParams['axes.prop_cycle'].by_key()['color']

color_label_map = {
    label: palette[i % len(palette)]
    for i,label in enumerate(labels_dict)
}


fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

plot_labels = ['lep_pt', 'lep_eta', 'n_jets', 'H_T']

for i, key in enumerate(plot_labels):
    for label in labels_dict:
        hep.histplot(histograms[key][label], bins=bins_dict[key], label=label, ax=axes[i], linewidth=1.5, color=color_label_map[label])
        hep.histplot(histograms_var_pT_res[key][label], bins=bins_dict[key], label=f"{label} pt_scale_up", ax=axes[i], linewidth=1.5, linestyle='--', color=color_label_map[label])

    axes[i].set_yscale('log')
    axes[i].set_xlabel(xlabel_dict[key], size=16)
    axes[i].set_ylabel('Events', size=16)
    axes[i].legend()

plt.tight_layout()
plt.show()
